# Quantum Tic-Tac-Toe

In [6]:
# Goal: Build a game of Tic-Tac-Toe where moves can be superpositions, and the board
# collapses when measured, using quantum game logic, entanglement, and measurement

"""
- Each move is a qubit in superposition between "X --> |1>" and "O --> |0>"
- Measuring a tile collapses it to 1 value
- Quantum strategies (entangled moves, Grover-based tile selection, quantum randomness in moves)
- Start with 3x3 board, each cell is a qubit, simulate quantum moves
- Add entangled gameplay (2 tiles share an entangled state --> move in 1 tile influences result in another)
    Example of this:
    qc.h(0)
    qc.cx(0, 1) 
    This entangles positions 0 and 1 in a Bell state
"""

'\n- Each move is a qubit in superposition between "X --> |1>" and "O --> |0>"\n- Measuring a tile collapses it to 1 value\n- Quantum strategies (entangled moves, Grover-based tile selection, quantum randomness in moves)\n- Start with 3x3 board, each cell is a qubit, simulate quantum moves\n- Add entangled gameplay (2 tiles share an entangled state --> move in 1 tile influences result in another)\n    Example of this:\n    qc.h(0)\n    qc.cx(0, 1) \n    This entangles positions 0 and 1 in a Bell state\n'

In [29]:
#winning system:
import numpy as np

def win_detection(post):
    game_won = False
    a = 0
    b = 0
    c = 0
    d = 2

    while a < 7 and game_won == False:
        if post[a] == post[a + 1] == post[a + 2]:
            game_won = True
        else:
            a = a + 3
    while b < 3 and game_won == False:
        if post[b] == post[b + 3] == post[b + 6]:
            game_won = True
        else:
            b = b + 1
    if post[c] == post[c + 4] == post[c + 7]:
        game_won = True
    if post[d] == post[d + 2] == post[d + 4]:
        game_won = True
    
    if game_won == True:
        # print("\n Game over")
        i = 0
        # while i < 9:
        #     print(post[i], post[i + 1], post[i + 2])
        #     i = i + 3
        return game_won

In [30]:
#simulate the circuit
from qiskit import transpile, QuantumCircuit
from qiskit_aer import Aer

def simulation(qc):
    backend = Aer.get_backend('aer_simulator')
    job = backend.run(transpile(qc, backend), shots = 1)
    result = job.result()
    counts = result.get_counts()
    outcome = list(counts.keys())[0]  #like '11' or '00'
    return outcome

In [41]:

#create TicTacToe blueprint:
class QuantumTicTacToe:

    def __init__(self):
        self.board = [' ' for _ in range(9)]
        self.entangled_pairs = {}

    #set up the board:
    def display(self):
        print("\n")
        for i in range(0, 9, 3):
            print(f" {self.board[i]} | {self.board[i + 1]} | {self.board[i + 2]}")
            if i < 6:
                print("---|---|---")

    #generic tile move:
    def quantum_move(self):
        qc1 = QuantumCircuit(1, 1)
        qc1.h(0)  # Superposition: equal chance of X or O
        qc1.measure(0, 0)
        outcome = simulation(qc1)
        return outcome

    #entangle positions a and b:
    def entangle(self, a, b):
        if self.board[a] == ' ' or self.board[b] == ' ':
            print("1 or both tiles are unfilled \n")
            return
        #make a dict with a and b --> {a, b}:
        self.entangled_pairs[0] = a
        self.entangled_pairs[1] = b

        self.collapse_entangled_pair(a, b)

    #create entangled Bell pair:
    def collapse_entangled_pair(self, a, b):
        qc2 = QuantumCircuit(2, 2)
        qc2.h(0)
        qc2.cx(0, 1) #Bell state
        qc2.measure_all() #collapse a and b
        outcome = simulation(qc2) #like '11' or '00'

        val_a = 'X' if outcome[1] == '1' else 'O' #if a collapses to 1  --> a = 'X' vice versa
        val_b = 'X' if outcome[0] == '1' else 'O'
        print(val_a)
        print(val_b)

        #change tiles a and b accordingly:
        self.board[a] = val_a 
        self.board[b] = val_b

    #make move:
    def make_move(self, tile):
        if self.board[tile] != ' ':
            print("Tile already filled \n")
            return
        result = self.quantum_move()
        self.board[tile] = 'X' if result == '1' else 'O'
        return result
    
    def move_checker(self, array, result, tile):
        for i in range(0, 8):
            if i == tile: #array[0] = 1
                if result == '1':
                    array[i] = 10
                else:
                    array[i] = 0
        game_won = win_detection(array)
        return game_won

    def is_full(self):
        return all(cell != ' ' for cell in self.board)

In [42]:
#play the game:
game = QuantumTicTacToe()
game_won = False
array = np.array([1,2,3,4,5,6,7,8,9])

print("Welcome to Quantum Tic-Tac-Toe")
print("You can:")
print("- Choose tiles between 0 and 8 which land O or X with a 50-50 chance each")
print("- Entangle placed tiles")

while not game.is_full() or game_won == False:
    a = False
    game.display()
    move = input("Input 0 - 8 to place a tile, or entangle tile_1 tile_2): ")
    if move.startswith("entangle"):
        a = True
        _, a, b = move.split()
        game.entangle(int(a), int(b))
        print(f"You entangled tiles", a, "and", b)

    for i in range(0, 8):
        if move == str(i):
            a = True
            result = game.make_move(int(move))
            game_won = game.move_checker(array, result, int(move))
            print(f"You moved tile ", int(move))
            if game_won == True:
                break
            break

    if a == False:
        print("Invalid input")
        break

if game.is_full() or game_won == True:
    game.display()
    print("\n Game over")

Welcome to Quantum Tic-Tac-Toe
You can:
- Choose tiles between 0 and 8 which land O or X with a 50-50 chance each
- Entangle placed tiles


   |   |  
---|---|---
   |   |  
---|---|---
   |   |  
You moved tile  0


 X |   |  
---|---|---
   |   |  
---|---|---
   |   |  
You moved tile  1


 X | X |  
---|---|---
   |   |  
---|---|---
   |   |  
O
O
You entangled tiles 0 and 1


 O | O |  
---|---|---
   |   |  
---|---|---
   |   |  
Invalid input
